# Dynamics 365 Business Central Trouble Shooting Guide (TSG) - Login issues (SaaS)

This notebook contains Kusto queries that can help getting to the root cause of a login issue for an environment in the online version of Business Central (SaaS). Each section in the notebook contains links to the TSG part of the authorization telemetry documentation in [aka.ms/bctelemetry](aka.ms/bctelemetry), as well as Kusto queries that help dive into a specific area. 

NB! The signal used in this notebook is only available in version 16.2 (or newer) of Business Central online, so check the version of your environment.

## 1\. Get setup: Load up Python libraries and connect to Application Insights

First you need to set the notebook Kernel to Python3, load the KQLmagic module (did you install it?) and connect to your Application Insights resource (get appid and appkey from the API access page in the Application Insights portal)

In [ ]:
# load the KQLmagic module
%reload_ext Kqlmagic

In [ ]:
# Connect to the Application Insights API
%kql appinsights://appid='<add app id from the Application Insights portal>';appkey='<add API key from the Application Insights portal>'

## 2\. Define filters

This workbook is designed for troubleshooting a single environment. Please provide values for aadTenantId and environmentName (or use a config file).

In [ ]:
# Add values for AAD tenant id, environment name, and date range.
# It is possible to leave the value for environment name blank (if you want to analyze across all values of the parameter)

# You can either use configuration file (INI file format) or set filters directly. 

# If you specify a config file, then variables set here takes precedence over manually set filter variables
# config file name and directory (full path)
configFile = "c:/tmp/notebook.ini"


# Add AAD tenant id and environment name here
aadTenantId = "MyaaDtenantId"
environmentName = ""
extensionId = "MyExtensionId"

# date filters for the analysis
# use YYYY-MM-DD format for the dates (ISO 8601)
startDate = "2020-11-20"
endDate = "2020-11-24"




# Do not edit this code section
import configparser
config = configparser.ConfigParser()

config.read(configFile)

if bool(config.defaults()):
    if config.has_option('DEFAULT', 'aadTenantId'):
        aadTenantId = config['DEFAULT']['aadTenantId']
    if config.has_option('DEFAULT', 'environmentName'):
        environmentName = config['DEFAULT']['environmentName']
    if config.has_option('DEFAULT', 'extensionId'):
        extensionId = config['DEFAULT']['extensionId']
    if config.has_option('DEFAULT', 'startDate'):    
        startDate = config['DEFAULT']['startDate']
    if config.has_option('DEFAULT', 'endDate'):
        endDate = config['DEFAULT']['endDate']

print("Using these parameters for the analysis:")
print("----------------------------------------")
print("aadTenantId         " + aadTenantId)
print("environmentName     " + environmentName)
print("startDate           " + startDate)
print("endDate             " + endDate)

# Analyze the login flow
Now you can run Kusto queries to look for possible root causes for login issues.

Either click **Run All** above to run all sections, or scroll down to the type of analysis you want to do and manually run queries

Documentation of the authentication and authorization flow: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/security/security-application

Kusto sample: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/Authorization.kql

## Authentication

Authentication in the online version of Business Central happens strictly in Azure Active Directory (AAD). Only when a user is authenticated in AAD, a session is attempted to be created in the Business Central server (NST). When dealing with login issues, check for *absense* of signal in eventIds for Authorization in the pre-open company of opening a session in the NST (eventIds RT0001 and RT0003) to determine if the issue is related to AAD (e.g. user is disabled, wrong password, failed MFA) or maybe something happening in the customer network (could be a DNS issue, or a changed firewall rule). 

**If you do not see any signal for eventIds RT0001 and RT0003, then start troubleshooting network issues first.**

Read more in the Security Guide here: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/security/security-application#authentication

In [ ]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and customDimensions.eventId in ('RT0001', 'RT0003' )
| summarize request_count=count() by bin(timestamp, 1h) | render timechart title= 'Number of pre-open company authorization attempts in the last day'

## Authorization failures (pre-open company)

A user can fail authorization before the open company trigger is executed for a number of different reasons:
* The user was successfully authenticated in Azure Active Directory but the user account is disabled in Business Central.
* A user successfully authenticated in Azure Active Directory but the user does not have any entitlements in Business Central (license issue)

Read more about these types of failures in the authorization signal docs here: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-authorization-trace#authorizationfailedpreopencompany

Kusto query: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/Authorization.kql

In [ ]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and customDimensions.eventId == 'RT0001'
| project timestamp
, guestUser = customDimensions.guestUser
, userType = customDimensions.userType
, failureReason = customDimensions.failureReason
, entitlementSetIds = customDimensions.entitlementSetIds
| order by timestamp desc
| limit 100

## Authorization failures (in the open company process)

Events show up here for a number of different reasons
* The company name is invalid
* User has no permission to access the company
* The environment is locked
* The license has expired or the trial period has ended
* The user's license is not valid for use on production companies

Read more in the authorization signal docs here: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-authorization-trace#authorization-failed-open-company

Kusto query: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/Authorization.kql


In [ ]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and customDimensions.eventId == 'RT0002'
| project timestamp
, clientType = customDimensions.clientType
, companyName = customDimensions.companyName
, failureReason = customDimensions.failureReason
| order by timestamp desc
| limit 100

## Successful logins (authentication in AAD succeded, authorization in the Business Central server succeeded)

If the user can authenticate against AAD and the two authorization steps inside the Business Central server succeeds, then a session is created and the user has successfully logged in.

Read more about application security in the Security Guide here:  https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/security/security-application#authentication

Kusto query: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/Authorization.kql

In [ ]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and customDimensions.eventId == 'RT0004'
| project timestamp
, clientType = customDimensions.clientType
, companyName = customDimensions.companyName
, totalTimeInMS = toreal(totimespan(customDimensions.totalTime))/10000 // totalTime is measured in ticks, divide by 10000 to get milliseconds
| order by timestamp desc
| limit 100

In [ ]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and customDimensions.eventId == 'RT0004'
| extend clientType = tostring( customDimensions.clientType )
| summarize count=count() by clientType, bin(timestamp, 1h)
| render timechart title= 'Number of successful logins the last day (shown by client/session type)'

In [ ]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and customDimensions.eventId in ('RT0001', 'RT0002', 'RT0004')
| extend attemptType = case(
    customDimensions.eventId == 'RT0001', 'Failure before open company' ,
    customDimensions.eventId == 'RT0002', 'Failure in open company trigger' ,
    customDimensions.eventId == 'RT0004', 'Successful login' , 
    'Unknown reason'
)
| summarize count=count() by attemptType, bin(timestamp, 1h)
| render timechart title= 'Number of login attempts the last day (shown by success/failure)'